
enum windows with cocoa?

clang -g -O0 -fmodules -x objective-c -fno-objc-arc -c main.c -o main.o

clang main.o -framework Cocoa -dynamiclib -rdynamic -o sog.dylib


In [ ]:

@import Cocoa;

static NSArray* windows;
static NSUInteger count;

#if __cplusplus
extern "C" {
#endif

__attribute__((visibility("default")))
int enumWindows()
{
	CGWindowListOption listOptions = kCGWindowListOptionAll;
	listOptions |= kCGWindowListOptionOnScreenOnly;
	listOptions |= kCGWindowListExcludeDesktopElements;

	// Ask the window server for the list of windows.
	windows = (NSArray*)CGWindowListCopyWindowInfo(listOptions, kCGNullWindowID);
	count = [windows count];

	return count;
}

__attribute__((visibility("default")))
const char* getTitle(int i) {
	if(windows == NULL) {
		count	= enumWindows();
	}
	//for (NSUInteger i = 0; i < count; i++) {
	//}
	NSDictionary*   nswindowsdescription = [windows objectAtIndex:i];
	//NSNumber* windowid = (NSNumber*)[nswindowsdescription objectForKey:@"kCGWindowNumber"];
	NSString* windowName = (NSString*)[nswindowsdescription objectForKey:@"kCGWindowOwnerName"];
	NSNumber* windowLayer = (NSNumber*)[nswindowsdescription objectForKey:@"kCGWindowLayer"];
	NSString* res = [NSString stringWithFormat:@"%@: %s", windowLayer, [windowName UTF8String]];
	return (char *)[res UTF8String];
}


int main(int argc, char *argv[])
{
    int count = enumWindows();
		for(int i = 0; i < count; i++) {
			printf("%s\n", getTitle(i));
		}
}

#if __cplusplus
}
#endif




lookup window by?


In [ ]:
__attribute__((visibility("default")))
const char* getTitle(int i) {
    if (windows == NULL) {
        count = enumWindows();
    }

    NSDictionary* targetWindow = nil;
    for (NSDictionary* window in windows) {
        NSNumber* windowLayer = (NSNumber*)[window objectForKey:(__bridge NSString*)kCGWindowLayer];
        if (windowLayer && [windowLayer intValue] == i) { // Compare with the provided index
            targetWindow = window;
            break;
        }
    }

    if (!targetWindow) {
        return NULL;
    }

    NSString* windowName = (NSString*)[targetWindow objectForKey:(__bridge NSString*)kCGWindowOwnerName];
    NSNumber* windowLayer = (NSNumber*)[targetWindow objectForKey:(__bridge NSString*)kCGWindowLayer];
    NSString* res = [NSString stringWithFormat:@"%@: %s", windowLayer, [windowName UTF8String]];
    
    return strdup([res UTF8String]); // Use strdup to return a persistent C-string
}

